# Camada GOLD

## Informações Gerais
  | Information | Details |
  |---------|------|
  |Table Name | gold.pedidos|
  |Origem | silver.pedidos|

## Histórico de Atualizações
|Data | Desenvolvido por | Motivo |
|:------:|--------|-------|
|mm/dd/aaaa|Data Eng.|details|

In [0]:
### Import of libs

from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
### Nome do Esquema e Tabela no catálogo

database = 'gold'
tabela = 'venda_total_pedidos'

In [0]:
# Funções

## Função que aplica comentário na tabela!

def adicionaComentariosTabela(a,b,c,d):
    spark.sql(f"COMMENT ON TABLE {a}.{b} IS 'C'")
    for key,value in d.items():
        sqlaux = f"ALTER TABLE {a}.{b} CHANGE COLUMN {key} COMMENT '{value}'"
        spark.sql(sqlaux)

In [0]:
df_pedidos = spark.sql(
f"""
SELECT
    Produto AS Nome_produto,
    CAST(SUM(quantidade_vendida) AS INT) AS quantidade_total,
    CAST(SUM(Preco_Unitario) AS DECIMAL(20,2)) AS valor_total,
    CAST(SUM(Preco_Unitario*quantidade_vendida) AS DECIMAL(20,2)) AS faturamento_total
FROM
    silver.pedidos
GROUP BY all
""")

In [0]:
## Incluir colunas de controle

df_pedidos = df_pedidos.withColumn("data_carga", current_date())
df_pedidos = df_pedidos.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL 3 HOURS"))


In [0]:
## Na camada SILVER é necessário a existência de Metadados, para maior compreensão das colunas. 
#Isso pode ser feito de forma MANUAL ou automática

# para automatico, ver a função AdicionaComentariosTabela

comentario_tabela = "esta tabela é uma tabela corporativa para XXX usos"

lista_comentario_colunas ={
'Nome_produto' : 'chave do prodiuto',
'quantidade_total': 'Quantidade vendida na operação',
'faturamento_total': 'faturamento de vera',
'valor_total': 'soma faturamento'
}

print('Dados Gravados!')



Dados Gravados!


In [0]:
df_pedidos.write \
    .format('delta') \
    .mode('overwrite') \
    .clusterBy("nome_produto") \
    .option('mergeSchema','true') \
    .option('overwriteSchema','true') \
    .saveAsTable(f'{database}.{tabela}')
adicionaComentariosTabela(database,tabela,comentario_tabela,lista_comentario_colunas)
print("Dados gravados com sucesso")

Dados gravados com sucesso


In [0]:
spark.sql(f"OPTIMIZE {database}.{tabela}")
print("Otimização Finalizada")

Otimização Finalizada


In [0]:
%sql
--1 
SELECT *
FROM gold.venda_total_pedidos
ORDER BY quantidade_total DESC --Pra mim, n faz mto sentido comparar diferentes produtos por QUANTIDADE! Melhor faturamento!!!!
LIMIT 5;

Nome_produto,quantidade_total,valor_total,faturamento_total
Cerveja XYZ,15036,606.78,165041.51
Vinho ABC,14191,614.80,168253.57
Suco Natural,12931,444.03,126052.65
Água Mineral,12874,533.71,135812.69
Refrigerante 123,11741,459.77,129574.58


In [0]:
%sql
--1 
SELECT *
FROM gold.venda_total_pedidos
ORDER BY faturamento_total DESC 
LIMIT 5;

-- CHECAR COM O PATRICK: quando o meu DPM pede o maior produto em VENDAS, eu deveria olhar QTT*preço ou  apenas preço.?

Nome_produto,quantidade_total,valor_total,faturamento_total
Vinho ABC,14191,614.80,168253.57
Cerveja XYZ,15036,606.78,165041.51
Água Mineral,12874,533.71,135812.69
Refrigerante 123,11741,459.77,129574.58
Suco Natural,12931,444.03,126052.65
